# Generate HiC Heatmaps From Juicer Data
- **Author** - Frank Grenn
- **Date Started** - April 2020
- **Quick Description:** Convert juicer .hic files to .cool or .h5 files to generate heatmap figures with the [hicExplorer](https://hicexplorer.readthedocs.io/en/latest/content/tools/hicPlotMatrix.html#hicplotmatrix) tool

In [ ]:
import os

## Setup Environment for HiCExplorer Tool on Biowulf

In [ ]:
print("\
source /path/to/conda.sh\n\
conda activate base\n\
which python\n\
conda create -n HICExplorer_project python=3.7 \n\
conda activate HICExplorer_project\n\
conda config --env --add channels defaults\n\
conda config --env --add channels conda-forge\n\
conda config --env --add channels bioconda\n\
conda config --show-sources\n\
pip install hicexplorer\n\
      ")

In [ ]:
JUICER_DIR="/path/to/juicer"
PLOT_DIR="/path/to/plots"

In [ ]:
!mkdir {PLOT_DIR}

In [ ]:
#list of the sample names taken from the juicer directory
SAMPLES = sorted([ name for name in os.listdir(JUICER_DIR) if os.path.isdir(os.path.join(JUICER_DIR, name)) and 'HICS' in name ])
print(len(SAMPLES))
print(SAMPLES)


In [ ]:
for sample in SAMPLES:
    !mkdir {PLOT_DIR}/{sample}

In [ ]:
resolutions = [2500000, 1000000, 500000, 250000, 100000, 50000, 25000, 10000, 5000]
resolutions

In [ ]:
res_str = [str(r) for r in resolutions]
res_spaced = ' '.join(res_str)
print(res_spaced)

In [ ]:
AUTOSOMES = [str(x) for x in list(range(1,23))]
SEXOMES = ['X','Y']
CHROMOSOMES = AUTOSOMES + SEXOMES
print(CHROMOSOMES)

## convert the .hic files to .cool for each resolution

In [ ]:
   
with open(f"{PLOT_DIR}/hic_to_cool_samples.swarm","w") as swarm_file:
    for sample in SAMPLES:  
        script_dir = f"{PLOT_DIR}/{sample}/scripts"
        !mkdir {script_dir}
        with open(f"{script_dir}/hic_to_cool.sh","w") as conv_file:
            conv_file.write(f"#!/bin/bash\n\
module load python/3.7\n\
conda activate HICExplorer_project\n\
hicConvertFormat -m {JUICER_DIR}/{sample}/aligned/inter_30.hic --inputFormat hic --outputFormat cool -o {PLOT_DIR}/{sample}/inter_30.cool --resolutions {res_spaced}\n\
")
        conv_file.close()
        swarm_file.write(f"bash {script_dir}/hic_to_cool.sh\n")
swarm_file.close()

In [ ]:
print(f"swarm -f {PLOT_DIR}/hic_to_cool_samples.swarm -g 20 -t 10 --sbatch '--mail-type=ALL' --time=24:00:00")

## process the .cool files for plots

In [ ]:
norm_types = ['KR']#['KR', 'VC', 'VC_SQRT']


In [ ]:
with open(f"{PLOT_DIR}/plot_samples.swarm","w") as swarm_file:
    for sample in SAMPLES:
        
        script_dir = f"{PLOT_DIR}/{sample}/scripts"
        img_dir = f"{PLOT_DIR}/{sample}/images"
        !mkdir {script_dir}
        !mkdir {img_dir}
        for res in resolutions:
            res_dir = f"{img_dir}/{res}"
            !mkdir {res_dir}
            for chrm in CHROMOSOMES:
                for norm in norm_types:
                    with open(f"{script_dir}/chr{chrm}_{res}_{norm}_plot.sh","w") as plot_file:
                        plot_file.write(f"#!/bin/bash\n\
module load python/3.7\n\
conda activate HICExplorer_project\n\
hicConvertFormat -m {PLOT_DIR}/{sample}/inter_30_{res}.cool --inputFormat cool --outputFormat cool -o {PLOT_DIR}/{sample}/inter_30_{res}_{chrm}_{norm}.cool --chromosome {chrm} --correction_name {norm}\n\
hicPlotMatrix --title '{sample} Chr {chrm}' --scoreName 'KR Normalized Contacts' --perChromosome --matrix {PLOT_DIR}/{sample}/inter_30_{res}_{chrm}_{norm}.cool --log --outFileName {res_dir}/{sample}_{res}_{chrm}_{norm}_log.png\n\
rm {PLOT_DIR}/{sample}/inter_30_{res}_{chrm}_{norm}.cool")
                        
                    plot_file.close()
                    swarm_file.write(f"bash {script_dir}/chr{chrm}_{res}_{norm}_plot.sh\n")
    swarm_file.close()

In [ ]:
print(f"swarm -f {PLOT_DIR}/plot_samples.swarm -g 240 -t 10 --sbatch '--mail-type=ALL' --time=24:00:00")